In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pymc3 as pm
import math
import pandas as pd
import arviz as az

In [ ]:
T_home = np.arange(100)
N_home = np.arange(300)
TN_home = np.empty((100, 300))
R_home = np.empty((100, 300))
X_home = np.empty((100, 8, 300))
t_home = TN_home.copy()

In [ ]:
with pm.Model() as home_model:
    ha = pm.Normal("ha", mu=0, sigma=math.sqrt(2))
    at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100, 8))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_home)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta + ha)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_home) * theta, observed=R_home)

pm.model_to_graphviz(home_model)

In [ ]:
home_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/home.csv")

In [ ]:
# create test data
X_test = np.zeros((100, 8, 300))
## create X
for i, row in home_data.iterrows():
    if int(i / 100) >= 300:
        for x in range(len(row)):
            if x == 0 or x == 9:
                continue
            X_test[int(row[0])][x - 1][int(i / 100 - 300)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_test.shape

In [ ]:
trace = az.from_netcdf("/home/morten/Develop/Live-Win-Prob/models/home_trace.nc")

In [ ]:
pm.set_data({"data": X_test}, model=home_model)
ppc_test = pm.sample_posterior_predictive(trace, model=home_model, samples=1000)

In [ ]:
ppc_test["like"].shape

In [ ]:
ppc_only_actual_params = ppc_test["like"][:, :, :4]

In [ ]:
ppc_only_actual_params.shape

In [ ]:
ppc_only_params_game_one_home = ppc_only_actual_params[:, :, 0]
ppc_only_params_game_one_home.shape

In [ ]:
ppc_only_params_game_one_home[:, 0].shape

In [ ]:
# create plot

In [ ]:
T_away = np.arange(100)
N_away = np.arange(300)
TN_away = np.empty((100, 300))
R_away = np.empty((100, 300))
X_away = np.empty((100, 8, 300))
t_away = TN_home.copy()

In [ ]:
with pm.Model() as away_model:
    at = pm.Normal("at", mu=0, sigma=math.sqrt(2), shape=(100, 8))
    beta = pm.Normal("beta", mu=0, sigma=math.sqrt(2))

    data = pm.Data("data", X_away)
    theta = pm.invlogit((at[:, :, None] * data).sum(axis=1) + beta)
    # theta = pm.invlogit(pm.math.dot(at, xt_theano) + beta)

    like = pm.Poisson("like", (100 - t_away) * theta, observed=R_away)

pm.model_to_graphviz(away_model)

In [ ]:
away_data = pd.read_csv("/home/morten/Develop/Live-Win-Prob/data/away.csv")

In [ ]:
# create test data
X_test_away = np.zeros((100, 8, 300))
## create X
for i, row in away_data.iterrows():
    if int(i / 100) >= 300:
        for x in range(len(row)):
            if x == 0 or x == 9:
                continue
            X_test_away[int(row[0])][x - 1][int(i / 100 - 300)] = row[x]

# X = X.reshape(X.shape[0]*X.shape[1], X.shape[2])
X_test_away.shape

In [ ]:
trace_away = az.from_netcdf("/home/morten/Develop/Live-Win-Prob/models/away_trace.nc")

In [ ]:
pm.set_data({"data": X_test_away}, model=away_model)
ppc_test_away = pm.sample_posterior_predictive(
    trace_away, model=away_model, samples=1000
)

In [ ]:
ppc_only_actual_params_away = ppc_test_away["like"][:, :, :4]

In [ ]:
plt.hist(ppc_only_actual_params_away[:, 20, 0])

In [ ]:
game_num = 1

In [ ]:
ppc_only_actual_params_away_game = ppc_only_actual_params_away[:, :, game_num]
ppc_only_actual_params_away_game.shape
ppc_only_actual_params_home_game = ppc_only_actual_params[:, :, game_num]
ppc_only_actual_params_home_game.shape

In [ ]:
R_test_home = np.zeros((100, 300))
## create R
for i, row in home_data.iterrows():
    if int(i / 100) >= 300:
        R_test_home[int(row[0])][int(i / 100 - 300)] = int(row[9])
R_test_away = np.zeros((100, 300))
## create R
for i, row in away_data.iterrows():
    if int(i / 100) >= 300:
        R_test_away[int(row[0])][int(i / 100 - 300)] = int(row[9])

In [ ]:
game_probs = np.zeros((100, 3))
goal_home_idx = []
goal_away_idx = []
for i in range(100):
    hist_home = np.histogram(
        ppc_only_actual_params_home_game[:, i],
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        density=True,
    )
    hist_away = np.histogram(
        ppc_only_actual_params_away_game[:, i],
        bins=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        density=True,
    )
    goals_home = R_test_home[0][game_num] - R_test_home[i][game_num]
    goals_away = R_test_away[0][game_num] - R_test_away[i][game_num]
    if i > 0:
        if R_test_home[i][game_num] != R_test_home[i - 1][game_num]:
            goal_home_idx.append(i)
        if R_test_away[i][game_num] != R_test_away[i - 1][game_num]:
            goal_away_idx.append(i)
    goal_matrix = np.zeros((9, 9))
    home_prob = 0
    away_prob = 0
    draw_prob = 0
    for h in range(9):
        for a in range(9):
            goal_matrix[h][a] = hist_home[0][h] * hist_away[0][a]
            home = goals_home + h
            away = goals_away + a
            if home > away:
                home_prob += hist_home[0][h] * hist_away[0][a]
            elif away > home:
                away_prob += hist_home[0][h] * hist_away[0][a]
            elif home == away:
                draw_prob += hist_home[0][h] * hist_away[0][a]

    game_probs[i][0] = home_prob
    game_probs[i][1] = draw_prob
    game_probs[i][2] = away_prob

In [ ]:
plt.plot(game_probs[:, 0], color="green")
plt.plot(game_probs[:, 1], color="black")
plt.plot(game_probs[:, 2], color="red")
for goal in goal_away_idx:
    plt.axvline(goal, color="red", linestyle="--")
for goal in goal_home_idx:
    plt.axvline(goal, color="green", linestyle="--")

In [ ]:
X_test_2 = np.zeros((100, 8, 300))
for idx in range(len(X_test)):
    X_test_2[idx] = X_test[0]

In [ ]:
R_test = np.zeros((100, 300))
## create R
for i, row in home_data.iterrows():
    if int(i / 100) >= 300:
        R_test[int(row[0])][int(i / 100 - 300)] = int(row[9])

In [ ]:
pm.set_data({"data": X_test_2}, model=home_model)
ppc_test = pm.sample_posterior_predictive(trace, model=home_model, samples=1000)

In [ ]:
ppc_only_actual_params = ppc_test["like"][:, :, :]

In [ ]:
plt.hist(ppc_only_actual_params[:, 20, 0])

In [ ]:
plt.hist(ppc_only_actual_params[:, 20, 1])

In [ ]:
plt.hist(ppc_only_actual_params[:, 20, 0])

In [ ]:
plt.hist(ppc_only_actual_params[:, 20, 250])

In [ ]:
R_test = R_test[:, :4]

In [ ]:
game0 = ppc_only_actual_params[:, 0]
game1 = ppc_only_actual_params[:, 1]
game2 = ppc_only_actual_params[:, 2]
game3 = ppc_only_actual_params[:, 3]

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.plot(game0)
ax.plot(R_test[:, 0])

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.plot(game1)
ax.plot(R_test[:, 1])

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.plot(game2)
ax.plot(R_test[:, 2])

In [ ]:
fig = plt.figure()
ax = fig.subplots()

ax.plot(game3)
ax.plot(R_test[:, 3])